In [62]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,concat_ws

In [63]:
spark = SparkSession.builder.appName('Customer Review').getOrCreate()

In [64]:
df_input = spark.read.options(header='True', InferSchema='True').csv('customer_data.csv')
#df_input.show()

In [77]:
##finding out the time difference
df_customer_concat = df_input.withColumn('time_diff', col('endtime').cast("long")- col('starttime').cast("long"))
df_customer_concat = df_customer_concat.withColumn('Cus_CusCare', concat_ws('_','from_number','to_number')).withColumn('CusCare_Cus', concat_ws('_','to_number','from_number'))
#df_customer_concat.show()

In [78]:
## distributing dataset for both missed and non missed calls
df_missed_call = df_customer_concat.filter('time_diff == 0')
df_non_missed_call = df_customer_concat.filter('time_diff != 0')
df_missed_call = df_missed_call.withColumnRenamed("Cus_CusCare","Customer_CustomerCare")
df_missed_call = df_missed_call.withColumnRenamed("From_number","Customer_Call_Received")
df_missed_call = df_missed_call.withColumnRenamed("to_number","to")
df_missed_call = df_missed_call.withColumnRenamed("time_diff","td")
df_non_missed_call = df_non_missed_call.withColumnRenamed("CusCare_Cus","CustomerCare_Customer")
#df_missed_call.show()
#df_non_missed_call.show()

In [79]:
##Customer who received call within 30 mint
df_output = df_missed_call.join(df_non_missed_call, df_missed_call.Customer_CustomerCare == df_non_missed_call.CustomerCare_Customer, 'inner')
df_output = df_output.filter('time_diff < 1800').selectExpr('Customer_Call_Received')
df_output.show()

+----------------------+
|Customer_Call_Received|
+----------------------+
|                   100|
|                   200|
|                   300|
+----------------------+

